In [1]:
import itertools
import copy
from engine import SlotGame


def expected_bar_fill(game: SlotGame):
    """
    Computes expected bar fill per spin given current upgrades.
    """
    symbols = game.symbols
    base_probs = dict(game.symbol_probs)

    # Apply reel_bias if active
    if game.upgrades["reel_bias"]:
        reel_probs = base_probs.copy()
        if "D" in reel_probs:
            reel_probs["D"] += 0.05
        if "E" in reel_probs:
            reel_probs["E"] += 0.10
        if "A" in reel_probs:
            reel_probs["A"] = max(0.0, reel_probs["A"] - 0.10)
        if "B" in reel_probs:
            reel_probs["B"] = max(0.0, reel_probs["B"] - 0.05)
        total = sum(reel_probs.values())
        for s in reel_probs:
            reel_probs[s] /= total
    else:
        reel_probs = base_probs

    # Precompute multipliers
    multipliers = game.get_effective_multipliers()
    boost_factor = 1.5 if game.upgrades["bar_boost"] else 1.0

    expected = 0.0

    # Iterate over all possible rows
    for row in itertools.product(symbols, repeat=game.cols):
        # Probability of this row
        p_row = 1.0
        for sym in row:
            p_row *= reel_probs[sym]

        # Compute increment (like evaluate_spin, but without updating game state)
        symbol_counts = {s: row.count(s) for s in symbols}
        increment = 0.0
        matched = False
        for symbol, count in symbol_counts.items():
            if count == 3:
                increment = multipliers[symbol] * game.bar_fill_per_match.get("3_same", 0.0) * boost_factor
                matched = True
                break  # only one symbol can occupy all 3
            elif count == 2:
                increment = multipliers[symbol] * game.bar_fill_per_match.get("2_same", 0.0) * boost_factor
                matched = True
                break

        if not matched and game.bar_fill_per_match.get("1_same", 0.0) > 0:
            singles = [s for s, c in symbol_counts.items() if c == 1]
            if singles:
                best_symbol = max(singles, key=lambda s: multipliers[s])
                increment = multipliers[best_symbol] * game.bar_fill_per_match.get("1_same", 0.0) * boost_factor

        expected += p_row * increment

    return expected


def analyze_all_upgrades(config_dict):
    game = SlotGame(config_dict=config_dict)
    results = {}

    # all combinations of upgrades
    upgrade_names = list(game.upgrades.keys())
    for mask in range(1 << len(upgrade_names)):
        g = SlotGame(config_dict=config_dict)
        for i, name in enumerate(upgrade_names):
            g.upgrades[name] = bool(mask & (1 << i))
        exp_fill = expected_bar_fill(g)
        results[tuple(sorted([k for k, v in g.upgrades.items() if v]))] = exp_fill

    return results


# Example usage:
if __name__ == "__main__":
    import json
    with open("D://pythonprojects/game_math/slot-game-project-1/data/example_config.json", "r") as f:
        config = json.load(f)

    results = analyze_all_upgrades(config)
    for upgrades, exp in results.items():
        name = ", ".join(upgrades) if upgrades else "No upgrades"
        print(f"{name:40s} -> Expected bar fill per spin: {exp:.4f}")


No upgrades                              -> Expected bar fill per spin: 0.1606
reel_bias                                -> Expected bar fill per spin: 0.2208
extra_spins                              -> Expected bar fill per spin: 0.1606
extra_spins, reel_bias                   -> Expected bar fill per spin: 0.2208
bar_boost                                -> Expected bar fill per spin: 0.2410
bar_boost, reel_bias                     -> Expected bar fill per spin: 0.3312
bar_boost, extra_spins                   -> Expected bar fill per spin: 0.2410
bar_boost, extra_spins, reel_bias        -> Expected bar fill per spin: 0.3312
bonus_multiplier_upgrade                 -> Expected bar fill per spin: 0.1807
bonus_multiplier_upgrade, reel_bias      -> Expected bar fill per spin: 0.2434
bonus_multiplier_upgrade, extra_spins    -> Expected bar fill per spin: 0.1807
bonus_multiplier_upgrade, extra_spins, reel_bias -> Expected bar fill per spin: 0.2434
bar_boost, bonus_multiplier_upgrade      -> 